In [1]:

import pandas as pd
import numpy as np
import random
from itertools import combinations

# === Kullanıcı Parametreleri ===
CSV_PATH = "value_bets_20251920_gpt.csv"   # Kendi dosya yolunuzu girin
N_COUPONS = 100         # Üretilecek valid kupon sayısı
STAKE_MIN = 5         # Kupon stake eşiği
MAX_ATTEMPTS = 100000   # Valid kupon bulmak için maksimum deneme sayısı
RANDOM_SEED = None        # Rastgelelik tekrar üretilebilir olsun

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

EXPECTED_COLS = [
    "match_date", "match_time", "match_id", "hometeam", "awayteam",
    "bet_name", "probability", "odds"
]

def _to_float_odds(x):
    """Odds sütununda virgüllü yazımları ("1,15") 1.15'e çevirir."""
    if pd.isna(x):
        return np.nan
    s = str(x).strip().replace('"', '').replace("'", "")
    s = s.replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return np.nan

def kelly_fraction(p: float, odds: float) -> float:
    """Kupon düzeyinde Kelly oranı (f*). odds <= 1 ise 0 döndürür."""
    if pd.isna(p) or pd.isna(odds) or odds <= 1.0 or p <= 0.0 or p >= 1.0:
        return 0.0
    b = odds - 1.0
    q = 1.0 - p
    f_star = (b * p - q) / b
    return max(f_star, 0.0)

def compute_single_bet_metrics(df: pd.DataFrame) -> pd.DataFrame:
    """Referans amaçlı tekil metrikler (kupon hesabında kullanılmaz)."""
    out = df.copy()
    p = out["probability"] / 100.0
    b = out["odds"] - 1.0
    q = 1.0 - p

    out["EV"] = p * out["odds"]
    # Kelly tekil
    out["Kelly"] = np.where(b > 0, ((b * p - q) / b).clip(lower=0.0), 0.0)
    out["Kelly"] = out["Kelly"].fillna(0.0)
    out["stake"] = out["Kelly"] * 100.0

    out["bankroll_if_win"]  = 1.0 + out["Kelly"] * (out["odds"] - 1.0)
    out["bankroll_if_lose"] = 1.0 - out["Kelly"]
    out["expected_bankroll"] = p * out["bankroll_if_win"] + (1.0 - p) * out["bankroll_if_lose"]
    return out


In [2]:

# === 1) Veri Yükleme ve Temizlik ===
raw = pd.read_csv(CSV_PATH)

# Eğer fazladan kolonlar (örn. 'tuttu/yattı') varsa yalnızca beklenen kolonları al.
if set(EXPECTED_COLS).issubset(raw.columns):
    df = raw[EXPECTED_COLS].copy()
else:
    # İlk 8 kolonu al ve başlıkları beklenen isimlerle ayarla (sağlamlık için)
    df = raw.iloc[:, :len(EXPECTED_COLS)].copy()
    df.columns = EXPECTED_COLS

# Tip ve format düzeltmeleri
df["match_id"] = df["match_id"].astype(str)
df["probability"] = pd.to_numeric(df["probability"], errors="coerce")
df["odds"] = df["odds"].apply(_to_float_odds)

# Geçersiz satırları at
df = df.dropna(subset=["match_id", "probability", "odds"])
df = df[df["odds"] > 1.0]
df = df[(df["probability"] >= 0.0) & (df["probability"] <= 100.0)].copy()

print(f"Yüklendi: {len(df)} satır")
df.head(3)


Yüklendi: 8546 satır


,match_date,match_time,match_id,hometeam,awayteam,bet_name,probability,odds
0,2025-09-20,17:00:00,2301024,Levanger,STOR/BLINK,Hangi Yarıda Daha Fazla Gol Atılır? :: 1. Yarı,38.0,2.69
1,2025-09-21,20:30:00,2303595,B.Dortmund,Wolfsburg,Maç Sonucu :: MS X,25.0,4.25
2,2025-09-21,20:30:00,2303595,B.Dortmund,Wolfsburg,Maç Sonucu :: MS 2,24.0,4.77


In [3]:

# === 2) Tekil Bahis Metrikleri (Referans) ===
bets = compute_single_bet_metrics(df)
bets.head(5)


,match_date,match_time,match_id,hometeam,awayteam,bet_name,probability,odds,EV,Kelly,stake,bankroll_if_win,bankroll_if_lose,expected_bankroll
0,2025-09-20,17:00:00,2301024,Levanger,STOR/BLINK,Hangi Yarıda Daha Fazla Gol Atılır? :: 1. Yarı,38.0,2.69,1.0222,0.013136,1.313609,1.0222,0.986864,1.000292
1,2025-09-21,20:30:00,2303595,B.Dortmund,Wolfsburg,Maç Sonucu :: MS X,25.0,4.25,1.0625,0.019231,1.923077,1.0625,0.980769,1.001202
2,2025-09-21,20:30:00,2303595,B.Dortmund,Wolfsburg,Maç Sonucu :: MS 2,24.0,4.77,1.1448,0.038408,3.840849,1.1448,0.961592,1.005562
3,2025-09-21,20:30:00,2303595,B.Dortmund,Wolfsburg,"Karşılıklı Gol ve 2,5 Gol Alt/Üst :: 2,5 Alt v...",12.0,8.41,1.0092,0.001242,0.124157,1.0092,0.998758,1.000011
4,2025-09-21,20:30:00,2303595,B.Dortmund,Wolfsburg,İlk Yarı / Maç Sonucu :: 1/2,3.0,34.50,1.0350,0.001045,0.104478,1.0350,0.998955,1.000037


In [4]:

# === 3) Kupon (3'lü) Üretimi ve Filtre ===

def generate_valid_coupons(bets_df: pd.DataFrame,
                           n_coupons: int = 100,
                           stake_min: float = 5.0,
                           max_attempts: int = 100000,
                           seed: int = 42):
    rng = np.random.default_rng(seed)
    valid = []
    seen_keys = set()
    attempts = 0

    # Index listesinden hızlı örnekleme
    idx = bets_df.index.to_list()

    while len(valid) < n_coupons and attempts < max_attempts:
        attempts += 1

        # 3 satır örnekle (replace=False) — her birinin match_id'si farklı olmalı
        sample_idx = rng.choice(idx, size=3, replace=False)
        sample = bets_df.loc[sample_idx]

        if sample["match_id"].nunique() < 3:
            continue

        # Kupon düzeyi olasılık ve oran
        p_coupon = (sample["probability"] / 100.0).prod()
        odds_coupon = sample["odds"].prod()

        # Kupon EV, Kelly, stake ve bankroll metrikleri (tamamen kupon bazında)
        EV_coupon = p_coupon * odds_coupon
        kelly_coupon = kelly_fraction(p_coupon, odds_coupon)
        stake_coupon = kelly_coupon * 100.0

        bankroll_if_win  = 1.0 + kelly_coupon * (odds_coupon - 1.0)
        bankroll_if_lose = 1.0 - kelly_coupon
        expected_bankroll = p_coupon * bankroll_if_win + (1.0 - p_coupon) * bankroll_if_lose

        # Valid koşulu
        if stake_coupon > stake_min and kelly_coupon > 0:
            # Duplike kuponları engelle (aynı üç maçtan oluşan)
            key = tuple(sorted(sample["match_id"].tolist()))
            if key in seen_keys:
                continue
            seen_keys.add(key)

            valid.append({
                "match_ids": list(key),
                "matches": sample.apply(lambda r: f"{r['hometeam']} - {r['awayteam']}", axis=1).tolist(),
                "bet_names": sample["bet_name"].tolist(),
                "component_probs": (sample["probability"] / 100.0).round(6).tolist(),
                "component_odds": sample["odds"].round(6).tolist(),
                "coupon_probability": round(p_coupon, 8),
                "coupon_odds": round(odds_coupon, 6),
                "coupon_EV": round(EV_coupon, 6),
                "coupon_Kelly": round(kelly_coupon, 6),
                "coupon_stake": round(stake_coupon, 4),
                "coupon_bankroll_if_win": round(bankroll_if_win, 6),
                "coupon_bankroll_if_lose": round(bankroll_if_lose, 6),
                "coupon_expected_bankroll": round(expected_bankroll, 6),
            })

    return valid, attempts

valid_coupons, attempts_used = generate_valid_coupons(
    bets, n_coupons=N_COUPONS, stake_min=STAKE_MIN, max_attempts=MAX_ATTEMPTS, seed=RANDOM_SEED
)
print(f"{len(valid_coupons)} valid kupon bulundu. Deneme sayısı: {attempts_used}")
len(valid_coupons)


95 valid kupon bulundu. Deneme sayısı: 100000


95

In [5]:
import json

# === 4) Sonuçları Kaydet ===
out_df = pd.DataFrame(valid_coupons)

# Liste kolonlarını JSON stringe çevirerek CSV'ye daha kullanılabilir biçimde yaz
list_cols = ["match_ids", "matches", "bet_names", "component_probs", "component_odds"]
for c in list_cols:
    if c in out_df.columns:
        out_df[c] = out_df[c].apply(lambda x: json.dumps(x, ensure_ascii=False))

OUTPUT_PATH = "valid_coupons_oley.csv"
out_df.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")
print(f"Kaydedildi: {OUTPUT_PATH} — {len(out_df)} satır")
out_df.head(5)


Kaydedildi: valid_coupons_oley.csv — 95 satır


,match_ids,matches,bet_names,component_probs,component_odds,coupon_probability,coupon_odds,coupon_EV,coupon_Kelly,coupon_stake,coupon_bankroll_if_win,coupon_bankroll_if_lose,coupon_expected_bankroll
0,"[""2300276"", ""2300770"", ""2303483""]","[""Bath City - Chelmsford C"", ""Elche - Real Ovi...","[""Toplam Gol Tek/Çift :: Tek"", ""Hangi Yarıda D...","[0.67, 0.38, 0.61]","[1.68, 2.96, 1.89]",0.155306,9.398592,1.459658,0.054730,5.4730,1.459658,0.945270,1.025157
1,"[""2300450"", ""2303037"", ""2303302""]","[""Independiente - San Lorenzo"", ""Tanjong Pagar...","[""Toplam Gol Alt/Üst :: 2,5 Üst"", ""Deplasman G...","[0.55, 0.73, 0.74]","[2.87, 1.37, 1.57]",0.297110,6.173083,1.834085,0.161236,16.1236,1.834085,0.838764,1.134484
2,"[""2300055"", ""2305961"", ""2392844""]","[""Gneist - Foerde"", ""Millwall - Watford"", ""JS ...","[""Toplam Gol Alt/Üst :: 3,5 Alt"", ""Karşılıklı ...","[0.66, 0.44, 0.7]","[1.94, 2.33, 1.46]",0.203280,6.599492,1.341545,0.060996,6.0996,1.341545,0.939004,1.020833
3,"[""2300831"", ""2341976"", ""2392844""]","[""JS Kairouanaise - ES Zarzis"", ""Kwun Tong - W...","[""Toplam Gol Alt/Üst :: 1,5 Üst"", ""Toplam Gol ...","[0.72, 0.63, 0.61]","[1.45, 1.79, 1.96]",0.276696,5.087180,1.407602,0.099727,9.9727,1.407602,0.900273,1.040649
4,"[""2300798"", ""2303357"", ""2305944""]","[""Sochi - CSKA"", ""CA Tigre - Aldosivi"", ""Braun...","[""Toplam Gol Alt/Üst :: 2,5 Üst"", ""Karşılıklı ...","[0.63, 0.59, 0.36]","[1.7, 2.24, 3.23]",0.133812,12.299840,1.645866,0.057157,5.7157,1.645866,0.942843,1.036916
